In [ ]:
import os
import sys
import boto3


s3client = boto3.client('s3')

In [ ]:
def create_presigned_url(bucket_name, object_name, expiration=28800):
    try:
        response = s3client.generate_presigned_url('get_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name},
                                                    ExpiresIn=expiration)
    except ClientError as e:
        logging.error(e)
        return None
    return response

**Specify the locations of the cloned [single-cell-curation repo](https://github.com/chanzuckerberg/single-cell-curation) & your API key file**

In [ ]:
scc_repo_loc = os.path.expanduser('~/GitClones/CZI/')
api_key_file_path = os.path.expanduser('~/Documents/keys/cxg-api-key.txt')

In [ ]:
sys.path.append(os.path.abspath(scc_repo + 'single-cell-curation/notebooks/curation_api/python/'))


from src.utils.config import set_api_access_config
from src.dataset import create_dataset,upload_local_datafile,upload_datafile_from_link
from src.collection import get_collection,create_revision

In [ ]:
set_api_access_config(api_key_file_path) #use env='dev' to work on the dev site

**Specify the Collection to upload to**<br>
If a Revision, use the Revision ID, not the Published ID

In [ ]:
collection_id = ''

In [ ]:
coll_datasets = {d['dataset_id']:d['title'] for d in get_collection(collection_id)['datasets']}
coll_datasets

**Specify which Dataset to upload to, or specify `new` to add a Dataset**

In [ ]:
d = ''

**Specify the bucket, any path directories (as prefix)**

In [ ]:
bucket = 'submissions-lattice'
prefix = 'cellxgene-curation/'

results = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter="/")

In [ ]:
for file in results.get('Contents'):
    if file.get('Size') == 0:
        continue
    if file.get('Key').endswith('_revised.h5ad'):
        print(file.get('Key'))

In [ ]:
file_key = 'cellxgene-curation/integrated_HBCA_cellxgene_wtissuetype_revised.h5ad'

url = create_presigned_url(bucket, file_key)
url

In [ ]:
if d == 'new':
    d = create_dataset(collection_id)
upload_datafile_from_link(url, collection_id, d)